# VAE+CNN Model Over Generated Lattices (9x9x9)

We need json and pandas to open up our data that was generated by spinglass_metropolis.c++ and format for our Model

In [ ]:
import json
import pandas as pd
import numpy as np

In [ ]:
# f = open('data.json')

# data = json.load(f)

In [ ]:
# # Prepare an empty list to store the pairs
# pairs = []

The for loop below is meant to label each lattice that is held witihn our data

In [ ]:
# # Iterate over the configurations
# for config in data['Configuration'].values():
#     for s in config.values():
#         for temp, lattice_list in s['Temp'].items():
#             # Convert the temperature to a float and determine the label
#             temp = float(temp)
#             label = 1 if temp < 1.1 else 0

#             # Iterate over each 9x9x9 array in the lattice list
#             for lattice in lattice_list:
                
#                 # Add the pair to the list
#                 pairs.append((temp, lattice, label))

Save the reformatted data into a pandas frame...

In [ ]:
# # Convert the list of pairs to a DataFrame
# df = pd.DataFrame(pairs, columns=['Temperature', 'Lattice', 'Label'])

# # Save the DataFrame to a CSV file
# df.to_csv('output.csv', index=False)

In [ ]:
# load the data frame.
df = pd.read_csv('output.csv')

In [ ]:
df

In [ ]:
import ast

# Convert the 'Lattice' column from string representation of list to numpy array, ensure values are float32 for tensorflow...
df['Lattice'] = df['Lattice'].apply(lambda x: np.array(ast.literal_eval(x)).astype('float32'))

# Get unique temperatures
temperatures = df['Temperature'].unique()

# Create a dictionary of DataFrames for each temperature
dfs = {temp: df[df['Temperature'] == temp] for temp in temperatures}


In [ ]:
dfs

# TRAIN_TEST_SPLIT DATA

NOTE: Tensorflow wont work normally with our X_train, X_test, if we were to split our data without a bit of further processing because its an array of arrays. We need to make the training be in a stack and add another column so that the library can properly make tensors

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np

In [ ]:
# Create a dictionary to hold the train and test splits for each temperature
splits = {}

In [ ]:
# Split the data

for temp in temperatures:
    # Get the lattices for this temperature
    X = dfs[temp]['Lattice'].values

    # Split the data into training and test sets
    X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)

    # Store the splits in the dictionary
    splits[temp] = (X_train, X_test)
    
    # Convert lists to numpy arrays and add an extra dimension for the 'channels' in our VAE+CNN
    X_train = np.stack(X_train)[..., np.newaxis]
    X_test = np.stack(X_test)[..., np.newaxis]

    # Store the splits in the dictionary
    splits[temp] = (X_train, X_test)

In [ ]:
for temp in temperatures:
    X_train, X_test = splits[temp]
    print("Here is the sizes of train and test for this temperature: ", temp)
    print(X_train.shape)
    print(X_test.shape)

# STRUCTURE OF VAE+CNN

In [ ]:
import tensorflow as tf
from tensorflow import keras

layers = keras.layers

class VAE(tf.keras.Model):
    def __init__(self, latent_dim):
        super(VAE, self).__init__()
        self.latent_dim = latent_dim
        self.encoder = self.create_encoder()
        self.decoder = self.create_decoder()

    def create_encoder(self):
        inputs = layers.Input(shape=(9, 9, 9, 1))
        x = layers.Conv3D(32, 3, activation="selu", strides=2, padding="same")(inputs)
        x = layers.Conv3D(64, 3, activation="selu", strides=2, padding="same")(x)
        x = layers.Flatten()(x)
        x = layers.Dense(16, activation="selu")(x)
        z_mean = layers.Dense(self.latent_dim, name="z_mean")(x)
        z_log_var = layers.Dense(self.latent_dim, name="z_log_var")(x)
        return tf.keras.Model(inputs, [z_mean, z_log_var], name="encoder")

    def create_decoder(self):
        latent_inputs = layers.Input(shape=(self.latent_dim,))
        x = layers.Dense(128 * 3 * 3 * 3, activation="selu")(latent_inputs)  # Increase the size here
        x = layers.Reshape((3, 3, 3, 128))(x)  # Now the total size matches
        x = layers.Conv3DTranspose(64, 3, activation="selu", strides=2, padding="valid")(x)
        x = layers.Conv3DTranspose(32, 3, activation="selu", strides=1, padding="valid")(x)
        decoder_outputs = layers.Conv3DTranspose(1, 3, activation="sigmoid", padding="same")(x)
        return tf.keras.Model(latent_inputs, decoder_outputs, name="decoder")

    
    def reparameterize(self, mean, log_var):
        eps = tf.random.normal(shape=mean.shape)
        return eps * tf.exp(log_var * .5) + mean

    def call(self, inputs):
        z_mean, z_log_var = self.encoder(inputs)
        z = self.reparameterize(z_mean, z_log_var)
        reconstructed = self.decoder(z)
        reconstructed = reconstructed * 2 - 1 #scale the output to be in range in the range [-1,1]
        # Add KL divergence regularization loss
        kl_loss = -0.5 * tf.reduce_mean(z_log_var - tf.square(z_mean) - tf.exp(z_log_var) + 1)
        self.add_loss(kl_loss)
        return reconstructed



# Lets Train!

In [ ]:
# Initialize a dictionary to hold the trained VAEs
vaes = {}

for temp, (X_train, X_test) in splits.items():
    # Initialize a VAE
    vae = VAE(latent_dim=3)

    # Compile the VAE
    vae.compile(optimizer='nadam', loss=tf.keras.losses.BinaryCrossentropy())

    # Train the VAE
    vae.fit(X_train, X_train, epochs=10, batch_size=32)

    # Store the trained VAE
    vaes[temp] = vae


In [ ]:
# Initialize dictionaries to hold the means and variances
means = {}
variances = {}

for temp, vae in vaes.items():
    # Use the encoder to transform the lattices into the latent space
    z_mean, z_log_var = vae.encoder.predict(splits[temp][0])  # Use the training data

    # Compute the mean and variance of the Gaussian distribution
    means[temp] = np.mean(z_mean, axis=0)
    variances[temp] = np.mean(np.exp(z_log_var), axis=0)

In [ ]:
import matplotlib.pyplot as plt

# Plot the means
plt.figure(figsize=(10, 5))
for i in range(len(list(means.values())[0])):  # Loop over each dimension of the latent space
    plt.plot(list(means.keys()), [m[i] for m in means.values()], marker='o', label=f'Mean Dimension {i+1}')
plt.title('Mean of the Gaussian Distribution as a Function of Temperature')
plt.xlabel('Temperature')
plt.ylabel('Mean')
plt.legend()  # Add a legend
plt.show()

# Plot the variances
plt.figure(figsize=(10, 5))
for i in range(len(list(variances.values())[0])):  # Loop over each dimension of the latent space
    plt.plot(list(variances.keys()), [v[i] for v in variances.values()], marker='o', label=f'Variance Dimension {i+1}')
plt.title('Variance of the Gaussian Distribution as a Function of Temperature')
plt.xlabel('Temperature')
plt.ylabel('Variance')
plt.legend()  # Add a legend
plt.show()


In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot(111, projection='3d')

# Get the temperatures
temps = list(means.keys())

# Get the means for each dimension
variance_dim1 = [m[0] for m in variances.values()]
varience_dim2 = [m[1] for m in variances.values()]
variance_dim3 = [m[2] for m in variances.values()]

# Create a 3D scatter plot
ax.scatter(temps, means_dim1, means_dim2, c=means_dim3)

ax.set_xlabel('Temperature')
ax.set_ylabel('Mean Dimension 1')
ax.set_zlabel('Mean Dimension 2')
plt.title('Varience of the Gaussian Distribution as a Function of Temperature')
plt.show()


# Validation?
above we saw that there was a behavior that was picked up within the lattices but how do we know if this is not just coincidence? Lets try to redo it but this time shuffling our data and seeing if we still see something similar?

In [ ]:
from sklearn.utils import shuffle

# Number of different shuffles you want to try
N = 5

# Initialize a list to hold all the VAEs
all_vaes = []

# Initialize a list to hold the train and test splits made to later extract the latent variables...
all_splits = []

for i in range(N):
    # Initialize a dictionary to hold the train and test splits for each temperature
    splits = {}
    # Initialize a dictionary to hold the trained VAEs
    vaes = {}

    for temp in temperatures:
        # Get the lattices for this temperature
        X = dfs[temp]['Lattice'].values

        # Shuffle the data
        X = shuffle(X, random_state=i)

        # Split the data into training and test sets
        X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)

        # Convert lists to numpy arrays and add an extra dimension for the 'channels' in our VAE+CNN
        X_train = np.stack(X_train)[..., np.newaxis]
        X_test = np.stack(X_test)[..., np.newaxis]

        # Store the splits in the dictionary
        splits[temp] = (X_train, X_test)

        # Initialize a VAE
        vae = VAE(latent_dim=3)

        # Compile the VAE
        vae.compile(optimizer='nadam', loss=tf.keras.losses.BinaryCrossentropy())

        # Train the VAE
        vae.fit(X_train, X_train, epochs=10, batch_size=32)

        # Store the trained VAE
        vaes[temp] = vae

    # Add the trained VAEs to the list
    all_vaes.append(vaes)
    all_splits.append(splits)

In [ ]:
# Initialize lists to hold the means and variances for all VAEs
all_means = []
all_variances = []

for i in range(N):
    # Get the VAEs and splits for this shuffle
    vaes = all_vaes[i]
    splits = splits_list[i]

    # Initialize dictionaries to hold the means and variances
    means = {}
    variances = {}

    for temp, vae in vaes.items():
        # Use the encoder to transform the lattices into the latent space
        z_mean, z_log_var = vae.encoder.predict(splits[temp][0])  # Use the training data

        # Compute the mean and variance of the Gaussian distribution
        means[temp] = np.mean(z_mean, axis=0)
        variances[temp] = np.mean(np.exp(z_log_var), axis=0)

    # Add the means and variances to the lists
    all_means.append(means)
    all_variances.append(variances)


In [ ]:
import matplotlib.pyplot as plt
import os

# Create a directory to save the plots
os.makedirs("plots", exist_ok=True)

# Loop over each shuffle
for i in range(N):
    # Get the means and variances for this shuffle
    means = all_means[i]
    variances = all_variances[i]

    # Plot the means
    plt.figure(figsize=(10, 5))
    for j in range(len(list(means.values())[0])):  # Loop over each dimension of the latent space
        plt.plot(list(means.keys()), [m[j] for m in means.values()], marker='o', label=f'Mean Dimension {j+1}')
    plt.title(f'Mean of the Gaussian Distribution as a Function of Temperature (Shuffle {i+1})')
    plt.xlabel('Temperature')
    plt.ylabel('Mean')
    plt.legend()  # Add a legend
    plt.savefig(f"plots/means_shuffle_{i+1}.png")  # Save the plot
    plt.close()

    # Plot the variances
    plt.figure(figsize=(10, 5))
    for j in range(len(list(variances.values())[0])):  # Loop over each dimension of the latent space
        plt.plot(list(variances.keys()), [v[j] for v in variances.values()], marker='o', label=f'Variance Dimension {j+1}')
    plt.title(f'Variance of the Gaussian Distribution as a Function of Temperature (Shuffle {i+1})')
    plt.xlabel('Temperature')
    plt.ylabel('Variance')
    plt.legend()  # Add a legend
    plt.savefig(f"plots/variances_shuffle_{i+1}.png")  # Save the plot
    plt.close()